In [3]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State


import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps
import base64

from crud import AnimalShelter


###########################
# Data Manipulation / Model
###########################
username = "aacuser"
password = "aac-123"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.read({}))


#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

# Image variables
image_filename = 'grazioso_salvare_logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard'))),
    html.Center(html.B(html.H1('Evan Bush Project 2 Dashboard'))),
    html.Hr(),
    html.Center(html.Img(id='customer-image',src='data:image/png;base64,{}'.format(encoded_image.decode()),alt='customer image')),
    html.Hr(),
    html.Div(
        dcc.RadioItems(
            id='filter-type',
            options=[
                {'label': 'Water Rescue', 'value': 'wr'},
                {'label': 'Mountain Rescue', 'value': 'mr'},
                {'label': 'Disaster Rescue', 'value': 'dr'},
                {'label': 'Reset', 'value': 'reset'},
            ],
        )),
    html.Hr(),
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        # Data table customization options
        data=df.to_dict('records'),
        editable=False,
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable=True,
        row_deletable=False,
        selected_columns=[],
        selected_rows=[],
        page_action="native",
        page_current= 0,
        page_size= 10,      
    ),
    html.Br(),
    html.Hr(),
    #This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])


#############################################
# Interaction Between Components / Controller
#############################################
  
@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
    # Water Rescue filter
    if filter_type == 'wr':
        df = pd.DataFrame.from_records(shelter.read({
                "animal_type": "Dog",
                "breed": {"$in": ["Labrador Retriever Mix","Chesa Bay Retr Mix", "Newfoundland Mix"]},
                "sex_upon_outcome": "Intact Female",
                "age_upon_outcome_in_weeks": {"$gte":26.0, "$lte":156.0}
        }))
    # Mountain filter
    elif filter_type == 'mr':
        df = pd.DataFrame.from_records(shelter.read({
                "animal_type": "Dog",
                "breed": {"$in": ["German Shepard","Alaskan Malamute","Old English Sheepdog", 
                                  "Siberian Husky", "Rottweiler"]},
                "sex_upon_outcome": "Intact Male",
                "age_upon_outcome_in_weeks": {"$gte":26.0, "$lte":156.0}
        }))
    # Disaster Rescue filter
    elif filter_type == 'dr':
        df = pd.DataFrame.from_records(shelter.read({
                "animal_type": "Dog",
                "breed": {"$in": ["Doberman Pinscher","German Shepard","Golden Retriever", 
                                  "Bloodhound","Rottweiler"]},
                "sex_upon_outcome": "Intact Male",
                "age_upon_outcome_in_weeks": {"$gte":20.0, "$lte":300.0}
        }))
        # Reset to no filter
    else:
        df = pd.DataFrame.from_records(shelter.read({}))
        
    columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
    data=df.to_dict('records')
            
    return (data,columns)


@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    dff = pd.DataFrame.from_dict(viewData)
    # Creates the values and names from the assigned data table
    names = dff['breed'].value_counts().keys().tolist()
    values = dff['breed'].value_counts().tolist()

    return [
        dcc.Graph(            
            figure = px.pie(dff, values=values, names=names, title='Available Dogs by Breed', width=1000, height=500)
        )    
    ]


@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):
    dff = pd.DataFrame.from_dict(viewData)
    pos_curser = [dff.iloc[0,13], dff.iloc[0,14]] # Map curser for lat and long of first document in filtered query

    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=pos_curser, zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            dl.Marker(position=pos_curser, children=[
                dl.Tooltip(dff.iloc[0,4]),
                dl.Popup([
                    html.H1('Animal Name'),
                    html.P(dff.iloc[0,9])
                ])
            ])
        ])
    ]


app